# FISP Twitter Projects Notebook

This notebook contains the latest code for pulling tweets and cleaning, manipilating the dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# coding: utf-8

# import necessary python packages
import sys
import tweepy #https://github.com/tweepy/tweepy
import csv
import time
import os
from collections import defaultdict
import logging
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from bs4 import BeautifulSoup

sys.path.insert(1, 'drive/My Drive/tweets_data_collection')
# Twitter and Dropbox API credentials
import api_cred as ac

In [0]:
# setup debug logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [0]:
# modify print precison for easier debugging
np.set_printoptions(precision=20)

# Helper Functions

In [0]:
# Gets all the authentication to start scraping from Twitter
def authenticate_twitter():
    auth = tweepy.OAuthHandler(ac.consumer_key, ac.consumer_secret)
    auth.set_access_token(ac.access_key, ac.access_secret)
    # Waiting important because if you're doing a fresh scrape, there
    # will be too many requests sent through Tweepy and it'll crash
    # Keep this to be True or suffer the consequences : - (
    api = tweepy.API(auth, wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True)
    return api

In [0]:
# Possible tags found in the json data per tweet
possible_info = {'contributors', 'coordinates', 'created_at', 'entities',
                 'favorite_count', 'favorited', 'full_text', 'geo', 'id',
                 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
                 'in_reply_to_status_id_str', 'in_reply_to_user_id',
                 'in_reply_to_user_id_str', 'is_quote_status', 'lang',
                 'place', 'possibly_sensitive', 'retweet_count',
                 'retweeted', 'retweeted_status', 'source', 'text', 'truncated',
                 'user'}

def get_new_tweets(tweet_name, since_id=1, api=authenticate_twitter(),
                   extended_mode=True,
                   info_tags={'id', 'created_at', 'full_text', 'retweet_count',
                              'favorite_count'}):
    """
    Grabs all the tweets since the last tweet loaded. Since_id will be the id
    of the last tweet. When grabbing tweets, we can only grab a max of 200 at a
    time. You can see this in the code below, and to go around that, we use
    "max-id", which will be the maximum id of the latest tweet loaded at a time.
    Every tweet posted incrementally is larger, so we maintain the "max_id" and
    get the tweets in reverse from the latest one up until our last taken tweet.
    i.e. If since_id == 4 and the latest tweets were from 300, we would get the
    tweets from 300 - 101, then 100 - 5. And the functionality of grabbing the
    tweets excludes the since_id.
    params: tweet_name - Twitter handle
            since_id - ID of the latest tweet seen so far
            api - Authentification of Twitter Account
            info_tags - json tags desired to keep track of
    """
    # Check if we can access account
    try:
        if api.get_user(tweet_name).protected:
            print('Tried to get from @%s, but account protected.' % (tweet_name))
            return None
    except tweepy.TweepError as e:
        print('Tried to get from @%s, but got a "%s" error' % (tweet_name, e))
        return None
        
    curr_info = possible_info.intersection(info_tags)
    curr_info.add('full_text')
    curr_info.add('text')
    if extended_mode:
        curr_info.remove('text')
        tweet_mode = 'extended'
    else:
        curr_info.remove('full_text')
        tweet_mode = 'compatibility'
    curr_info.add('id') # Necessary regardless
    curr_info = sorted(curr_info)
    tweets = np.array([])
    # Max number of tweets at a time is 200
    new_tweets = api.user_timeline(screen_name = tweet_name,
                                   since_id = since_id,
                                   count=200,
                                   tweet_mode = tweet_mode
                                   )
    while new_tweets:
        new_tweets = get_tweet_info(new_tweets, curr_info)
        tweets = np.append(tweets, new_tweets)
        # Max id
        max_id = new_tweets[-1]['id'] - 1
        new_tweets = api.user_timeline(screen_name = tweet_name,
                                       since_id = since_id,
                                       max_id = max_id,
                                       count = 200,
                                       tweet_mode = tweet_mode
                                       )

    logger.info("Downloading %d tweets from %s" % (len(tweets), tweet_name))
    # Reverses back to chronological order
    return list(tweets[::-1])

def get_necessary_info(tweet_status, info_list):
    res_info = {}
    for key in info_list:
        if key != 'full_text' or 'retweeted_status' not in tweet_status:
            res_info[key] = tweet_status[key]
        else:
            res_info[key] = 'RT @{}: {}'.format(tweet_status['retweeted_status']['user']['screen_name'],
                             tweet_status['retweeted_status'][key])
    return res_info

def get_tweet_info(tweet_statuses, info_list):
    """
    Grabs all the necessary json info and places them into a nice little dict
    """
    tweet_statuses = pd.Series(tweet_statuses).apply(lambda x: x._json)
    curr_tweets_series = tweet_statuses.apply(get_necessary_info, args=(info_list,))
    return curr_tweets_series.to_numpy()

# Write to Sheets ↓

The functions below write data to the currently local sheets.

## Scrape Tweets and save them to respective excel file

*twitter_list.xlsx* contains the list of candidates for each tweets excel file along with the metadata.

*cand_tweets.xlsx* contains the tweets for all announced candidates

*spec_tweets.xlsx* contains the tweets for all the speculated candidates

*rep_tweets.xlsx* contains the tweets for all current CA represenatives of interest for the CA 2018 Elections.

In [0]:
def df_to_file(row, out_fp):
    """
    Writes each row directly to file
    params: row - Twitter Handle and list of tweets
            out_fp - file output path
    """
    # Edge case where first row is repeated in apply function
    global first_row
    if first_row:
        first_row = False
        return
    
    curr_handle = row['handle']
    # Edge case when no tweets found
    if not row['tweet_list']:
      return {'handle': curr_handle,
              'since_id': 1}

    # Converts to dataframe with each tag and Twitter handle as a column
    curr_handle_tweets = pd.DataFrame(row['tweet_list'])
    curr_cols = curr_handle_tweets.columns.tolist()
    curr_handle_tweets['handle'] = curr_handle

    # Rearranges so that handle is the first column
    curr_handle_tweets = curr_handle_tweets[['handle'] + sorted(curr_cols)]

    # Appends to file
    curr_handle_tweets.to_csv(out_fp, mode='a', header=False, index=False)

    # Returns Twitter handle and most recent pulled id
    return {'handle': curr_handle,
            'since_id': curr_handle_tweets['id'].iloc[-1]}

In [0]:
def collect_data(tweet_output_fp, twitter_handles_fp, handles_record_fp,
                 tags={'id', 'created_at', 'full_text', 'retweet_count', 'favorite_count'},
                 chunk=100):
    """
    Pulls new latest tweets and appends them to the correct csv file
    params: tweet_output_fp - path to the file to save the new tweets 
            twitter_handles_fp - path to file that contains the twitter handles,
                                 last tweet pulled id, tweet counts, and
                                 last pull date
            handles_record_fp - path to record the last requested tweet
            tags - json tags to be stored into csv
            chunk - how many accounts we scrape at a time
    """

    global first_row

    # start timer
    start = time.time()
    logger.info("Start...")

    # process the paths so they are passable to load_sheets
    tweets_path = os.path.expanduser(tweet_output_fp)
    twitter_list_path = os.path.expanduser(twitter_handles_fp)
  
    # load and prepare list of twitter accounts
    list_df = pd.read_csv(twitter_list_path)
    
    # API Key
    api = authenticate_twitter()

    # Writes headers of output and records files
    with open(tweet_output_fp, 'w+') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['handle'] + sorted(tags))
    with open(handles_record_fp, 'w+') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['handle', 'id'])

    already_seen = set()
    for col in list_df.columns:
        if 'Twitter Handle' in col:
            for split in range(0, list_df.shape[0], chunk):
                curr_handles = list_df[col].iloc[split:split + chunk].dropna()
                if len(already_seen.intersection(curr_handles) - {' '}):
                    print('Intersection',
                          already_seen.intersection(curr_handles) - {' '})
                # Gets only the handles not seen yet
                curr_handles = pd.Series(list(set(curr_handles) - already_seen))
                if not curr_handles.shape[0]:
                    continue
                # Gets tweets
                curr_tweets_series = curr_handles.apply(get_new_tweets,
                                                        args=(1, api, True, tags))
                # Convert to format for df_to_file
                curr_tweets_series = curr_tweets_series.rename('tweet_list')
                curr_tweets_series.index = curr_handles
                curr_tweets_df = curr_tweets_series.reset_index()
                curr_tweets_df.rename(columns={'index': 'handle'}, inplace=True)
                # first_row to avoid repeating first row              
                first_row = True
                max_id_series = curr_tweets_df.apply(df_to_file,
                                                 args=(tweets_path,),
                                                 axis=1)
                max_id_series[0] = df_to_file(curr_tweets_df.iloc[0],
                                          tweets_path)
                # Get the max_id for the new since_id value
                max_id_df = pd.DataFrame(max_id_series.tolist()).drop_duplicates()
                max_id_df.to_csv(handles_record_fp, mode='a',
                                 header=False, index=False)
                already_seen = already_seen.union(curr_handles)
                print('Done up to %s' % (curr_handles.iloc[-1]))

#### Tweets Pull Variables and Function Executions 

Go here if you are looking to just run the code, make sure you have imported and set the correct setting above before running these cells

In [0]:
data_dir = 'drive/My Drive/tweets_data_collection/'

state_leg_tweets_output = 'complete_state_leg_tweets.csv'
state_leg_tweets_record = 'complete_state_leg_handles_record.csv'
state_leg_twitter_handles = 'State Legislator Twitter Handles - Lower House Full Sample.csv'

In [10]:
collect_data(tweet_output_fp=data_dir + state_leg_tweets_output,
             twitter_handles_fp=data_dir + state_leg_twitter_handles,
             handles_record_fp=data_dir + state_leg_tweets_record,
             chunk=100)

INFO:__main__:Start...
INFO:__main__:Downloading 121 tweets from kevinpayneLD21
INFO:__main__:Downloading 525 tweets from jkt_ak
INFO:__main__:Downloading 3182 tweets from Rios_Rebecca
INFO:__main__:Downloading 3217 tweets from LivingstonLD22
INFO:__main__:Downloading 28 tweets from BenallyLD7
INFO:__main__:Downloading 379 tweets from RepAndyJ
INFO:__main__:Downloading 0 tweets from RepZachFansler
INFO:__main__:Downloading 2908 tweets from alaskascott
INFO:__main__:Downloading 771 tweets from JDMesnard


Tried to get from @CathyTilton, but got a "[{'code': 63, 'message': 'User has been suspended.'}]" error


INFO:__main__:Downloading 527 tweets from Rep_LancePruitt
INFO:__main__:Downloading 50 tweets from AZDavidStringer
INFO:__main__:Downloading 0 tweets from AkRepJohnston
INFO:__main__:Downloading 156 tweets from SpeakerChenault
INFO:__main__:Downloading 1997 tweets from HeatherCarterAZ
INFO:__main__:Downloading 373 tweets from TammieKWilson
INFO:__main__:Downloading 2360 tweets from Barton4Az
INFO:__main__:Downloading 24 tweets from RepKnopp
INFO:__main__:Downloading 686 tweets from mattclaman
INFO:__main__:Downloading 170 tweets from PhilLovas
INFO:__main__:Downloading 2005 tweets from CesarChavezAZ
INFO:__main__:Downloading 578 tweets from RepLedoux1
INFO:__main__:Downloading 3228 tweets from anthonykernAZ
INFO:__main__:Downloading 1048 tweets from ericdescheenie
INFO:__main__:Downloading 1446 tweets from PaulMosleyAZ
INFO:__main__:Downloading 3191 tweets from charissemillett
INFO:__main__:Downloading 968 tweets from DeLenaAK
INFO:__main__:Downloading 0 tweets from repsaddler
INFO:__m

Tried to get from @RepSamKitoIII, but account protected.


INFO:__main__:Downloading 43 tweets from RepDanOrtiz


...

Tried to get from @SergioMunozJr, but account protected.
Done up to SergioMunozJr
Intersection {'RepDanielRiemer', 'repsteffen', 'CortezPhilip', 'DavidCrowleyWI', 'SarahforHD134', 'JohnJagler', 'helengiddings', 'RolandForTexas', 'charliegeren', 'RepSchofield', 'tittlassembly', 'DaleKooyenga', 'RepGoyke', 'JohnsonForTexas', 'RepHorlacher', 'LindaKoopTX', 'AlmaAllen131', 'RepJanel', 'GoldmanCraig', 'JFarrarDist148', 'Robert_Hutton', 'RepChrisSinicki', 'NicoleCollier95', 'tomasuresti', 'RepDonVruwink', 'RepJessie', 'MattRinaldiTX', 'JasonVillalba', 'RepThierry146', 'RepJRod', 'AnaHdzTx', 'SpeakerStraus', 'Ron_Tusler', 'dwayne_bohac', 'RafaelAnchia', 'GeneforTexas', 'MaryFelzkowski', 'jarvisjohnsontx', 'GFColeman', 'Jbrostoff', 'MorganMeyerTX', 'RepKessler', 'vote4ina', 'AdamJarchow28', 'BriscoeCain', 'KleefischJoel', 'VoteBarbara', 'RepKitchens', 'CarolforTexas', 'TomOliverson', 'rodneyanderson', 'DanHuberty', 'Bill_Zedler', 'sweak1', 'jimsteineke', 'TonyDaleTX', 'SZ4WI', 'RepLyleLarson',

INFO:__main__:Downloading 67 tweets from DanKnodlforWi
INFO:__main__:Downloading 269 tweets from voteamywi
INFO:__main__:Downloading 114 tweets from VorpagelFor27


Done up to VorpagelFor27
Intersection {'RepRobBrooks', 'kmbernier1', 'MikeKuglitsch', 'GordonHintz', 'RepHebl', 'Thiesfeldt', 'derekstanford', 'Tranel4Assembly', 'skrug75', 'JesseForWI', 'AdamNeylon', 'paulgraveswa', 'RepRohrkaste', 'RepKloba', 'pike_liz', 'MayorCoryMason', 'davemurphy56', 'VoteToddNovak', 'repvos', 'Rep_Taylor', 'Rep_Jimmy', 'RepStuckWi', 'scote_allen', 'brandonvick', 'RepJillBillings', 'PeterWBarca', 'VickiKraft', 'RepTodOhnstad', 'steve4house', 'RepDeBolt', 'ElectLillian', 'BethDoglio', 'Strom_Peterson', 'RepQuinn', 'SherryAppleton2', 'RepSondy', 'RepMattShea', 'BobKulp', 'rep89', 'marcusricceli', 'jtwilcox111', 'rob_summerfield', 'electginamccabe', 'RepWarrenPetryk', 'AKB1968', 'RepSargent', 'TereseBerceau', 'NickMilroy', 'votechandler', 'RepHesselbein', 'BradKlippert', 'ZackHudgins', 'LisaSubeck', 'RepShankland', 'repschraa', 'samanthakerkman', 'RepSnyder85', 'repmarydye', 'ChrisTaylorWI', 'DanaJWachs'}


INFO:__main__:Downloading 51 tweets from Chuck4Assembly
INFO:__main__:Downloading 1300 tweets from johnmacco
INFO:__main__:Downloading 91 tweets from Jspiros
INFO:__main__:Downloading 207 tweets from MarkForAssembly
INFO:__main__:Downloading 13 tweets from Duchow4Assembly


Done up to Duchow4Assembly
Intersection {'stokesbary', 'KristineLytton', 'MiaGregerson33', 'tinaorwall', 'gruberobinson', 'runsinthehalls', 'SteveTharinger', 'jessynfarrell1', 'NoelFrame', 'RepChapman', 'JohnLovickForWa', 'MichelleCaldier', 'senpattykuderer', 'RepDrewhansen', 'StateRepKilduff', 'jakefey', 'pettigrew_e', 'lauriejinkins2', 'RepGoodman', 'eileencody', 'DickMuri', 'LuanneVanWerven', 'DrewMacEwen', 'GaelTarleton', 'RepSpringer', 'Sharonfor49th', 'cindy32ld', 'Koster4Congress', 'VincentBuys', 'M_Stambaugh_', 'votenicolemacri', 'RepJesseYoung', 'MarkHargrove', 'joefitzgibbon', 'markharm', 'DGGriffey', 'JoyceMcDonaldWA', 'TanaSenn', 'RepGerryPollet'}
Tried to get from @electkmreeves, but got a "[{'code': 50, 'message': 'User not found.'}]" error
Done up to electkmreeves


In [9]:
tester_df = pd.read_csv(data_dir + state_leg_tweets_output, 
                        names=['handle'] + sorted({'id', 'created_at', 'full_text', 'retweet_count', 'favorite_count'}))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
handles_df = pd.read_csv(data_dir + state_leg_tweets_record,
                         names=['handle', 'max_id'])

In [11]:
dup_mask = tester_df.handle.value_counts() > 3248
print(tester_df.handle.value_counts()[dup_mask])
dup_handles = tester_df.handle.value_counts()[dup_mask].index

Belsito4CT          746977
Democrats ~ 100%    196580
DelegateHorn        103927
bobiteman            94525
JaredEMoskowitz      26346
AngieChenButton       8318
Name: handle, dtype: int64


In [12]:
for dup_handle in dup_handles:
    print(tester_df.loc[tester_df['handle'] == dup_handle]\
          .created_at.value_counts())

Fri Mar 17 11:42:09 +0000 2017    131044
Tue Aug 29 13:16:18 +0000 2017    130973
Sat Mar 11 13:58:16 +0000 2017    130778
Wed Oct 11 12:05:59 +0000 2017    130769
Thu Aug 03 13:43:32 +0000 2017    130727
                                   ...  
Sat Feb 03 17:27:59 +0000 2018         1
Thu Aug 03 15:46:34 +0000 2017         1
Sat Apr 07 13:57:59 +0000 2018         1
Mon Sep 25 14:26:11 +0000 2017         1
Tue Aug 08 15:20:41 +0000 2017         1
Name: created_at, Length: 1536, dtype: int64
Series([], Name: created_at, dtype: int64)
Thu Jul 24 04:22:07 +0000 2014    102832
Wed May 11 06:19:39 +0000 2016         2
Fri Mar 20 00:18:49 +0000 2015         2
Wed May 13 18:50:33 +0000 2015         2
Wed May 13 18:51:49 +0000 2015         1
                                   ...  
Thu Jul 17 15:53:12 +0000 2014         1
Sat Oct 15 23:44:04 +0000 2016         1
Tue Dec 06 19:33:03 +0000 2016         1
Wed Apr 06 19:57:20 +0000 2016         1
Fri Jul 08 06:28:14 +0000 2016         1
Name: crea

In [13]:
tester_df.drop_duplicates().handle.value_counts().loc[dup_handles]

Belsito4CT          1536
Democrats ~ 100%       1
DelegateHorn        1096
bobiteman           1958
JaredEMoskowitz     2216
AngieChenButton     1562
Name: handle, dtype: int64

In [14]:
print(tester_df.drop_duplicates().shape)

(3620089, 6)


In [15]:
tester_df.drop_duplicates().dropna().shape

(3619561, 6)

In [0]:
tester_df.drop_duplicates().dropna()\
    .to_csv(data_dir + 'complete_no_dups_state_leg_tweets.csv',
            index=False)

In [0]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from pytorch_pretrained_bert import BertAdam
from transformers import BertTokenizer, get_linear_schedule_with_warmup, BertConfig, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import datetime
import random
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [0]:
decoder_dict = {"‚Äù": '"',
                "‚Äú": '"',
                "‚Ä¶": '...',
                "‚Äô": "'",
                "‚Äò": "'",
                "‚Äì": "-",
                "‚Äî": "-"}
def cleaner(txt):
    """
    Custom cleaner of text
    """
    txt = txt.split()
    for i in range(len(txt)):
        word = txt[i]
        while 'Ä' in word:
            char_i = word.index('Ä')
            to_decode = word[char_i - 1: char_i + 2]
            if to_decode in decoder_dict:
                word = word[:char_i - 1] + decoder_dict[to_decode]\
                        + word[char_i + 2:]
            else:
                word = word[:char_i - 1] + " " + word[char_i + 2:]
        if word in ['t', 's', 'd', 'll', 've', 're', 'm']:
            txt[i] = txt[i - 1] + "'" + word
            txt[i - 1] = ''
            continue
        link_pattern = re.compile('htt.*[^\s]*')
        word = link_pattern.sub(' ', word)
        amp_pattern = re.compile('&amp;')
        word = amp_pattern.sub('and', word)
        alpha_num_pattern = re.compile("[^a-zA-Z0-9_.,:']+")
        word = alpha_num_pattern.sub(' ', word)
        txt[i] = word.strip()
    txt = ' '.join(txt).strip()
    if txt == '...' or txt == '':
        return np.NaN
    return txt

def bert_tokenize_f(sent, tokenizer):
    """
    Tokenize all of the sentences and map the tokens to thier word IDs.
        (1) Tokenize the sentence.
        (2) Prepend the `[CLS]` token to the start.
        (3) Append the `[SEP]` token to the end.
        (4) Map tokens to their IDs.
    """
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    return encoded_sent
  
def pad_sequences_f(token_arr, maxlen, dtype="long",
                    value=0, truncating="post", padding="post"):
    return pad_sequences([token_arr], maxlen=maxlen, dtype=dtype, 
                         value=value, truncating=truncating,
                         padding=padding)[0]

In [0]:
tester_df_new = tester_df.drop_duplicates().dropna()

In [0]:
tester_df_new['cleaned_text'] = tester_df_new['full_text'].apply(cleaner)
tester_df_new = tester_df_new.dropna()

In [29]:
tester_df_new['tokenized_text'] = tester_df_new['cleaned_text'].apply(bert_tokenize_f, args=(bert_tokenizer,))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
print('Max sentence length: ', max(tester_df_new['tokenized_text'].apply(len)))

Max sentence length:  280


### Additional Functions

In [0]:
def load_sheets(path):
    sheet_book = load_workbook(path)
    sheet_writer = pd.ExcelWriter(path, engine='openpyxl')
    sheet_writer.book = sheet_book
    sheet_writer.sheets = dict((ws.title, ws) for ws in sheet_book.worksheets)
    logger.info("Downloaded %s" % path)
    return sheet_writer

In [0]:
# Purpose: Updates like and retweet totals
def collect_addition_data(tweet_sheet, tweet_list, sheetname):
  # start the timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweet_sheet = os.path.expanduser(tweet_sheet)
  tweet_list = os.path.expanduser(tweet_list)
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(tweet_list)
  list_df = pd.read_excel(tweet_list, sheet_name=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(tweet_sheet)
  logger.info("Downloaded tweets list")
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for row in list_df.itertuples():  
    name, since_id, count = row[2], row[3],row[4]
    
    # check if rep has a twitter handle 
    if (type(name) is not unicode):
      continue
    
    # read cand tweet sheet
    tweets_df = pd.read_excel(tweet_sheet, sheetname=name)
    logger.info("Retrived data from spreadsheet for %s" % name)
    
    # retreive updated tweets
    tweets = get_new_tweets(name, 1)
    updates_df = pd.DataFrame(tweets)
    
    if (updates_df.empty):
      continue 
      
    # clean dataframe to only include id, retweets, and favorites
    updates_df = updates_df[[0, 6, 7]]
    updates_df.columns = ['id', 'retweets', 'favorites']
    
    # call helper fuction to match updated metadata with correct tweets
    tweets_df = update_metadata(tweets_df, updates_df, name)
    
    if (name == 'antonio4ca'):
      continue
      
    # write the updated data to the twitter profile's sheet to be saved
    tweets_df.to_excel(tweet_writer, sheet_name=name, index=False, startcol=1)
    logger.info("Updated data on spreadsheet for %s" % name)
    # 100 second pause between data pulls to avoid token exceptions
    time.sleep(20)
  
  tweet_writer.save()
  
  logger.info("Done collecting additional data")
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [0]:
# This function takes the up to date metadata and matches it to their respective tweet using a tweet's unique id
def update_metadata(tweets_df, updates_df, cand_name): 
  # convert tweet id to the same type as the updates sheet
  tweets_df['id'] = tweets_df['id'].astype(str)
  tweets_df.set_index('id', inplace=True)
  
  ## loop through the updates metadata and updates the tweet sheet
  for row in updates_df.itertuples():
    tweets_df.at[row[1], 'retweets'] = row[2]
    tweets_df.at[row[1], 'favorites'] = row[3]

  # drop null rows that could not match with a tweet
  tweets_df.dropna(subset=['created_at'], inplace=True)
  
  return tweets_df

In [0]:
#############################################################################################
# func that will pull a number of tweets and create a coding sheet with columns for rating
# sentitment, partisanship, policy, fact, and opinion
# params: coding_workbook - the path to the excel file containing the tweets
#         sheetname - sheet containing all of the tweets
#         sample_size - the size of the random sample to be taken from the full corpus of tweets
#         sample_name - the name of the sample used as the sheet name in the coding file
# returns: returns the samples dataframe – make sure to uncomment the portion that saves the sample if needed
def coding_sampling(coding_workbook, sheetname, sample_size, sample_name):
  logger.info("Start...")
  start = time.time()
  
  # properly load spreadsheet to append new data
  coding_workbook = os.path.expanduser(coding_workbook)
  sample_writer = load_sheets(coding_workbook)
  full_df = pd.read_excel(coding_workbook, sheetname)
  logger.info("Full data loaded")
  
  # the col headers and corresponding col dtype for each the sample sheet
  col_headers = {'id': [str],'text': [np.nan], 'sentiment': [np.nan], 'political': [np.nan], 'ideology': [np.nan],
                 'macroeconomics': [np.nan], 'immigration':[np.nan], 'national_security': [np.nan], 'ideology': [np.nan],
                 'crime_law_enforcement': [np.nan], 'civil_rights': [np.nan], 'environment': [np.nan],
                 'education': [np.nan], 'healthcare': [np.nan], 'governance': [np.nan], 'no_policy_content': [np.nan],
                 'asks_for_donation': [np.nan], 'ask_to_watch_read_share_follow_something': [np.nan], 'ask_etc': [np.nan],
                 'factual_claim': [np.nan], 'opinion': [np.nan]}
  # create dataframe to store the sample in using the col_headers dict
  sample_df = pd.DataFrame(col_headers)
  
  # keep only id and text 
  #full_df = full_df[['id', 'text']]
  
  # if there are existing samples in the workbook excludes the previously sampled data
  if( (len(sample_writer.sheets) > 1)):
    # get a list of the names of the sampled sheets, skip the first sheet since it contains all the data
    sampled_sheets = sample_writer.sheets
    sampled_sheets = (list(sampled_sheets.keys()))[1:]
    
    # store the col names to be used to remove temp cols 
    full_df_col = list(full_df)
    
    # loop thru the sheets and exclude previously tweets by id 
    # an issue may arise here once the sampled sheets include hand codings
    for sheet in sampled_sheets:
      sampled_df = pd.read_excel(coding_workbook, sheet) # read in prev sample
      # merge with full dataset with col indicating whether the data exists in both 
      cleaned_df = pd.merge(full_df, sampled_df, how='outer', on='id', indicator=True)
      # only keep the data that exists exclusively in full dataset
      cleaned_df = cleaned_df[cleaned_df['_merge'] == 'left_only']
      # drop the col indicating which dataset the entry belongs to and the text col belonging to the sample
      cleaned_df = cleaned_df.drop(['_merge', 'text_y'], axis=1)
      cleaned_df = cleaned_df.rename(index=str, columns={'text_x': 'text'}) # rename the orig text col 
      # set the full_df to cleaned_df and rearrange the col back to their original order
      full_df = cleaned_df
      full_df = full_df[full_df_col]
  
  # sample the tweets
  sample_tweets = full_df.sample(sample_size)
  logger.info("Sample made")
  
  # concat the sampled tweets with the coding measurments
  sample_df = pd.concat([sample_tweets, sample_df], sort=True)
  # reaarange the columns
  #sample_df = sample_df[['id', 'text', 'sentiment', 'political', 'ideology', 'macroeconomics', 'immigration',
  #                       'national_security', 'crime_law_enforcement', 'civil_rights', 'environment', 'education',
  #                       'healthcare', 'governance', 'no_policy_content', 'asks_for_donation',
  #                       'ask_to_watch_read_share_follow_something', 'ask_etc', 'factual_claim', 'opinion']]
  sample_df.drop(sample_df.tail(1).index,inplace=True) # drop last n rows
  logger.info("Sample processed and concated")
  
  # ensure id field is string to avoid excel float/sci rounding – uncomment this to save the sample to the workbook
  sample_df.id = sample_df.id.astype(str)
  sample_df.to_excel(sample_writer, sample_name, index=False)#, float_format='string')
  sample_writer.save()
  
  # testing the exlude portion
  if( (len(sample_writer.sheets) > 1)):
    # get a list of the names of the sampled sheets, skip the first sheet since it contains all the data
    sampled_sheets = sample_writer.sheets
    sampled_sheets = (sampled_sheets.keys())[1:0]
    full_df_col = list(full_df_col)
    failed_excludes = 0
    
    # loop thru the sheets and remove the tweets by id 
    for sheet in sampled_sheets:
      sampled_df = pd.read_excel(coding_workbook, sheet)
      cleaned_df = pd.merge(full_df, how='inner', on='id')
      failed_excludes = failed_excludes + len(cleaned_df.index)
    logger.info("The number of failed excludes: %d", failed_excludes)
      
  
  logger.info("done")
  return sample_df

In [0]:
df = coding_sampling(state_data_dir + coded_tweets, all_tweets, coding_sample_size, 'batch3')